In [1]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
import keras.utils as ku
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, InputLayer, Embedding, SimpleRNN, Dropout, LSTM, Bidirectional, Input, Conv1D, MaxPooling1D, Concatenate, Flatten
from keras import regularizers
from keras import metrics
from keras.activations import relu, softmax
from keras.losses import categorical_crossentropy
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


## Data Pre-processing

In [23]:
Y,X = [], []
lang=[]
lang_str =[]
cal_str =[]

def Preprocessing(f, num_lines):
    Y=[]
    X=[]
    lang=[]
    lang_str =[]
    cal_str =[]
    bad_list = ['https']
    for i in range(num_lines):
        if i%50000==0:
            print("Exampleno. :",i)
    #         print(line_d)
        line = f.readline()
        line_d = line.split()
        if len(line_d)==3:
            start=True
            Y.append(line_d[2])
        elif len(line_d)== 0 or len(line_d) == 1:
            pass
        elif line_d[0]=='@' or line_d[0]=='#':
            line = f.readline()
            pass
        elif len(line_d[0])==1:
            pass
        elif not line_d[0].isalpha():
            pass
        elif line_d[0]in bad_list:
            pass
        else:
            if line_d[-1]=='O':
                pass
            elif start:
                X.append(cal_str)
                lang.append(lang_str)
                cal_str, lang_str = [], []
                cal_str.append(line_d[0])
                lang_str.append(line_d[1])
                start=False
            else:
                cal_str.append(line_d[0])
                lang_str.append(line_d[1])
    return X, Y, lang
print('Train.txt')
f = open("raw_data/nlp_assignment_3/train.txt", "r",encoding="utf8")
test_num_lines = sum(1 for line in f)
f.close()

f = open("raw_data/nlp_assignment_3/train.txt", "r",encoding="utf8")
X, Y, lang = Preprocessing(f, test_num_lines)
f.close()

print('\n Test.txt')
f = open("raw_data/nlp_assignment_3/test.txt", "r",encoding="utf8")
test_num_lines = sum(1 for line in f)
f.close()

f = open("raw_data/nlp_assignment_3/test.txt", "r",encoding="utf8")
X_test, Y_test, lang_test = Preprocessing(f, test_num_lines)
f.close()


Train.txt
Exampleno. : 0
Exampleno. : 50000
Exampleno. : 100000
Exampleno. : 150000
Exampleno. : 200000
Exampleno. : 250000
Exampleno. : 300000
Exampleno. : 350000
Exampleno. : 400000

 Test.txt
Exampleno. : 0
Exampleno. : 50000


In [24]:
# correct Indexing
X.pop(0)
Y.pop(-1)
lang.pop(0)
lang_test.pop(0)
X_test.pop(0)
Y_test.pop(-1)
print("Training data:",len(X))
print("Test data:",len(X_test))

Training data: 15130
Test data: 1868


## Data Generation

In [25]:
# Forming sentences
sent_list = []
test_sent_list=[]
for line in X:
    sent_list.append(' '.join(line).lower())
for line in X_test:
    test_sent_list.append(' '.join(line).lower())

In [28]:
test_sent_list[:10]

['rt love looks good on maddie ako lang ba yung sobrang masaya kasi may zolo sya before with the past medyo lowkey',
 'ye ye ye we gonna start another june on sour note uhhhh yes no yes no yes',
 'of lyching mein kahna nae chahta qki mere yaha btay tco',
 'caring bohot jyada caring courier wale bsdk ke sign bhi khud hi krlete mera',
 'what nonesense kabhi baymani per bani team kamiyab nahi ho sakti jo log apnay liy co',
 'best of luck sir world cup ke liye bhot bhot subhkamnaye',
 'yes great dialogues in that one also chupke chupke over chhaddabeshi all except ke sholay co',
 'desh bhakti baat wahi samajh sakte hai jo khud deshbhakt wo log to sirf bharat tere tukde hong co rmdzsiwsgt',
 'pakistani team ne effort ki aagey allah ki marziiiiiiiiiiiii',
 'nsharif kiya tum apne baap ki oulad nahi kyun usy bay yaro madadgar chora hua kiya wo co']

In [80]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sent_list+ test_sent_list)
total_words = len(tokenizer.word_index) + 1
print("Size of word Index:", total_words)

Size of word Index: 34403


In [81]:
train_input_sequences = []
test_input_sequences = []

for line in sent_list:
    token_list = tokenizer.texts_to_sequences([line])[0]
    train_input_sequences.append(token_list)
for line in test_sent_list:
    token_list = tokenizer.texts_to_sequences([line])[0]
    test_input_sequences.append(token_list)

max_sequence_len = max(max([len(x) for x in train_input_sequences]), max([len(x) for x in test_input_sequences]))
train_input_sequences = np.array(pad_sequences(train_input_sequences, maxlen=max_sequence_len, padding='pre'))
test_input_sequences = np.array(pad_sequences(test_input_sequences, maxlen=max_sequence_len, padding='pre'))

# 001 -> Positive
# 010 -> Neutral
# 100 -> Negative
train_label =[]
test_label =[]
for i in Y:
    if i == 'negative':train_label.append([1,0,0])
    elif i == 'positive':train_label.append([0,0,1])
    else:train_label.append([0,1,0])
train_label = np.asarray(train_label)
print("Train Data prepared!")
for i in Y_test:
    if i == 'negative':test_label.append([1,0,0])
    elif i == 'positive':test_label.append([0,0,1])
    else:test_label.append([0,1,0])
test_label = np.asarray(test_label)
print("Test Data prepared!")

Train Data prepared!
Test Data prepared!


In [8]:
import json
# Source: Obtained from Github:precog-iiitd /mind-your-language-aaai
# Translation of some Hindi words (written in English) to English words
json1_file = open('dictionary_hinglish_transliteration.json')
hinglish = json1_file.read()
hinglish_dict = json.loads(hinglish)
hindi_words = hinglish_dict.keys()

# Source: Obtained from Github:HarshTrivedi /HinglishSentiment
Hindi_stopwords = ['ke', 'ka', 'ek', 'mein', 'ki', 'hai', 'yah', 'aur', 'se', 'hain', 'ko', 'par', 'iss', 'hota', 'jo', 'kar', 'me', 'gaya', 'karne', 'kiya', 'liye', 'apne', 'ne', 'bani', 'nahi', 'toh', 'hi', 'ya', 'avam', 'diya', 'ho', 'iska', 'tha', 'dhvara', 'hua', 'tak', 'saath', 'karna', 'vaale', 'baad', 'liya', 'aap', 'kuchh', 'sakte', 'kisi', 'ye', 'iska', 'sabse', 'ismein', 'the', 'do', 'hone', 'vah', 've', 'karte', 'bahut', 'kaha', 'varg', 'kai', 'karein', 'hoti', 'apni', 'unke', 'thi', 'yadi', 'hui', 'jaa', 'na', 'ise', 'kehte', 'kahte', 'jab', 'hote', 'koi', 'hue', 'va', 'abhi', 'jaise', 'sabhi', 'karta', 'unki', 'tarah', 'uss', 'aadi', 'kul', 'raha', 'iski', 'sakta', 'rahe', 'unka', 'issi', 'rakhein', 'apna', 'pe', 'uske']
count1=0
count=0
count2=0

for i in range(len(X)):
    for j in range(len(X[i])):
        text = X[i][j].lower()
        if text in hindi_words:
            count+=1
            X[i][j] = hinglish_dict[text]

print("Number of Hindi words replaced in train ",count)

count1=0
count=0
count2=0

for i in range(len(X_test)):
    for j in range(len(X_test[i])):
        text = X_test[i][j].lower()
        if text in hindi_words:
            count+=1
            X_test[i][j] = hinglish_dict[text]
        elif lang_test[i][j]=='Hin':
            count2+=1
            X_test[i][j] = ''        
print("Number of Hindi words replaced in test ",count)


# Glove Embeddings
EMBEDDING_DIM = 100
embeddings_index = {}
word_index = tokenizer.word_index
EMBEDDING_DIM = 100
glove_file = open('.glove.6B/glove.6B.100d.txt', encoding="utf8")

for line in glove_file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
glove_file.close()

print('Total word vectors: ',len(embeddings_index))

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


Number of Hindi words replaced in train  98728
Number of Hindi words replaced in test  12013
Total word vectors:  400000


## Architecture

In [70]:
def Custom_Model(max_len):
    sequence_input = Input(shape=(max_len,))
    embedded_sequences = Embedding(total_words, 100)(sequence_input)
    lstm1 = Bidirectional(LSTM(8,dropout=0.3,recurrent_dropout=0.3,return_sequences=True))(embedded_sequences)
    
    class_block, filter_sizes = [], [3,4,6,7]
    for diff_filters in filter_sizes:
        conv_l1 = Conv1D(filters=12,kernel_size=1,
                        activation='relu',)(lstm1)
        conv_l2 = Conv1D(filters=12,kernel_size=diff_filters,
                           activation='relu',kernel_regularizer=regularizers.l2(0.02))(conv_l1)
        class_block.append(conv_l2)

    merge_layer = Concatenate(axis=1)(class_block)
#     pool_1 = MaxPooling1D(3)(merge_layer)
    l_drop1 = Dropout(0.4)(merge_layer)
    flat1 = Flatten()(l_drop1)
    dense_l1 = Dense(24, activation='relu',kernel_regularizer=regularizers.l2(0.02))(flat1)

    out = Dense(3, activation='softmax')(dense_l1)
    model = Model(sequence_input, out)
    return model

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [71]:

custom_Model = Custom_Model(max_sequence_len)
# adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
# keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
custom_Model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc', recall_m, precision_m, f1_m])
custom_Model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 31)           0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, 31, 100)      3440300     input_12[0][0]                   
__________________________________________________________________________________________________
bidirectional_12 (Bidirectional (None, 31, 16)       6976        embedding_12[0][0]               
__________________________________________________________________________________________________
conv1d_89 (Conv1D)              (None, 31, 12)       204         bidirectional_12[0][0]           
___________________________________________________________________________________________

## Training

In [72]:
checkpoint = ModelCheckpoint('try_weights.h5',monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
n_epochs=8
history_custom = custom_Model.fit(train_input_sequences, train_label, batch_size=8, callbacks=callbacks_list, validation_data=(test_input_sequences, test_label), epochs=n_epochs, verbose=1)

Train on 15130 samples, validate on 1868 samples
Epoch 1/8
15130/15130 [==============================] - 82s 5ms/step - loss: 0.9819 - acc: 0.5454 - recall_m: 0.3231 - precision_m: 0.5857 - f1_m: 0.3997 - val_loss: 0.9689 - val_acc: 0.5589 - val_recall_m: 0.3542 - val_precision_m: 0.6046 - val_f1_m: 0.4374

Epoch 00001: val_loss improved from inf to 0.96886, saving model to try_weights.h5
Epoch 2/8
15130/15130 [==============================] - 80s 5ms/step - loss: 0.8299 - acc: 0.6408 - recall_m: 0.5632 - precision_m: 0.6729 - f1_m: 0.6086 - val_loss: 1.0331 - val_acc: 0.5691 - val_recall_m: 0.5064 - val_precision_m: 0.5876 - val_f1_m: 0.5409

Epoch 00002: val_loss did not improve from 0.96886
Epoch 3/8
15130/15130 [==============================] - 80s 5ms/step - loss: 0.7692 - acc: 0.6827 - recall_m: 0.6362 - precision_m: 0.7027 - f1_m: 0.6654 - val_loss: 0.9584 - val_acc: 0.5926 - val_recall_m: 0.5545 - val_precision_m: 0.6035 - val_f1_m: 0.5765

Epoch 00003: val_loss improved fro

## Scores

In [73]:
from sklearn.metrics import classification_report
y_pred = np.argmax(custom_Model.predict(test_input_sequences), axis=1)
report = classification_report(np.argmax(test_label, axis=1),y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.61      0.66      0.63       532
           1       0.58      0.50      0.54       754
           2       0.60      0.67      0.63       582

    accuracy                           0.60      1868
   macro avg       0.60      0.61      0.60      1868
weighted avg       0.60      0.60      0.59      1868

